<a href="https://colab.research.google.com/github/skssushil/TF-IDF-Cosine/blob/master/Represent_collection_of_documents_in_VSM_using_tf_and_tf_idf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math

In [ ]:
import io
with open('CISI.ALL') as f:
    lines = ""
    for l in f.readlines():
        lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    lines = lines.lstrip("\n").split("\n")

In [ ]:
doc_set = {}
doc_id = ""
doc_text = ""
for l in lines:
    if l.startswith(".I"):
        doc_id = l.split(" ")[1].strip()
    elif l.startswith(".X"):
        doc_set[doc_id] = doc_text.lstrip(" ")
        doc_id = ""
        doc_text = ""
    else:
        doc_text += l.strip()[3:] + " " # The first 3 characters of a line can be ignored.

In [ ]:
dataset = []
for i in doc_set.values():
  dataset.append(i)

In [ ]:
N=len(dataset)
print(N)

1460


In [ ]:
import io
with open('CISI.QRY') as g:
    qlines = ""
    for m in g.readlines():
        qlines += "\n" + m.strip() if m.startswith(".") else " " + m.strip()
    qlines = qlines.lstrip("\n").split("\n")

In [ ]:
qry_set = {}
qry_id = ""
qry_text=""
for m in qlines:
    if m.startswith(".I"):
        qry_id = m.split(" ")[1].strip()
    elif m.startswith(".W"):
        qry_set[qry_id] = m.strip()[3:]
        qry_id = ""
squery=qry_set["1"]

In [ ]:
def convert_lower_case(data):
    return np.char.lower(data)

In [ ]:

def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

In [ ]:
def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

In [ ]:
def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

In [ ]:
def stemming(data):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text

In [ ]:
# import the inflect library 
import inflect 
p = inflect.engine() 
  
# convert number into words 
def convert_number(text): 
    # split string into list of words 
    temp_str = text.split() 
    # initialise empty list 
    new_string = [] 
  
    for word in temp_str: 
        # if word is a digit, convert the digit 
        # to numbers and append into the new_string list 
        if word.isdigit(): 
            temp = p.number_to_words(word) 
            new_string.append(temp) 
  
        # append the word as it is 
        else: 
            new_string.append(word) 
  
    # join the words of new_string to form a string 
    temp_str = ' '.join(new_string) 
    return temp_str 

In [ ]:
def preprocess(data):
    data = convert_numbers(data)
    data = convert_lower_case(data)
    data = remove_punctuation(data) #remove comma seperately
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    data = stemming(data)
    data = remove_punctuation(data)
    data = convert_numbers(data)
    data = stemming(data) #needed again as we need to stem the words
    data = remove_punctuation(data) #needed again as num2word is giving few hypens and commas fourty-one
    data = remove_stop_words(data) #needed again as num2word is giving stop words 101 - one hundred and one
    return data

In [ ]:
processed_text = []
for i in dataset[:N]:
    processed_text.append(word_tokenize(str(preprocess(convert_number(i)))))

In [ ]:
DF = {}

for i in range(N):
    tokens = processed_text[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}
for i in DF:
    DF[i] = len(DF[i])

In [ ]:
total_vocab_size = len(DF)
print(total_vocab_size)

7004


In [ ]:
total_vocab = [x for x in DF]

In [ ]:
print(total_vocab[:20])

['eighteen', 'edit', 'dewey', 'decim', 'classif', 'comaromi', 'present', 'studi', 'histori', 'first', 'ddc', 'publish', '1876', 'eighteenth', '1971', 'futur', 'continu', 'appear', 'need', 'spite']


In [ ]:
def doc_freq(word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

In [ ]:
doc = 0
tf_idf = {}
for i in range(N):    
    tokens = processed_text[i]    
    counter = Counter(tokens)
    words_count = len(tokens)    
    for token in np.unique(tokens):        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1))        
        tf_idf[doc, token] = tf*idf
    doc += 1

In [ ]:
doc = 0
tf_idf = {}
for i in range(N):    
    tokens = processed_text[i]    
    counter = Counter(tokens)
    words_count = len(tokens)    
    for token in np.unique(tokens):        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1))        
        tf_idf[doc, token] = tf*idf
    doc += 1

In [ ]:
tf_idf[(0, 'abroad')]

0.10774739102985577

In [ ]:
len(tf_idf)

80132

In [ ]:
def matching_score(k, query):
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))
    print("Matching Score")
    print("\nQuery:", query)
    print("")
    print(tokens)    
    query_weights = {}
    for key in tf_idf:        
        if key[1] in tokens:
            try:
                query_weights[key[0]] += tf_idf[key]
            except:
                query_weights[key[0]] = tf_idf[key]    
    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse=True)
    print("")    
    l = []
    
    for i in query_weights[:10]:
        l.append(i[0])    
    print(l)

In [ ]:
matching_score(10, squery)

Matching Score

Query: What problems and concerns are there in making up descriptive titles? What difficulties are involved in automatically retrieving articles from approximate titles? What is the usual relevance of the content of articles to their titles?

['problem', 'concern', 'make', 'descript', 'titl', 'difficulti', 'involv', 'automat', 'retriev', 'articl', 'approxim', 'titl', 'usual', 'relev', 'content', 'articl', 'titl']

[1293, 448, 446, 314, 1280, 443, 428, 564, 721, 1162]


In [ ]:
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

In [ ]:
D = np.zeros((N, total_vocab_size))
for i in tf_idf:
    try:
        ind = total_vocab.index(i[1])
        D[i[0]][ind] = tf_idf[i]
    except:
        pass

In [ ]:
def gen_vector(tokens):
    Q = np.zeros((len(total_vocab)))
    counter = Counter(tokens)
    words_count = len(tokens)
    query_weights = {}
    for token in np.unique(tokens):   
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = math.log((N+1)/(df+1))
        try:
            ind = total_vocab.index(token)
            Q[ind] = tf*idf
        except:
            pass
    return Q

In [ ]:
def cosine_similarity(k, query):
    print("Cosine Similarity")
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))
    print("\nQuery:", query)
    print("")
    #print(tokens)
    d_cosines = []
    query_vector = gen_vector(tokens)
    for d in D:
        d_cosines.append(cosine_sim(query_vector, d))
    out = np.array(d_cosines).argsort()[-k:][::-1]
    print("")  
    print(out)
    print("Cosine similarity"," id ", " Document")
    for i in range(k):
      x=out[i]
      print(round(d_cosines[x],10),"    ", out[i]," ", dataset[x][:100])


In [ ]:
cosine_similarity(10,squery)

Cosine Similarity

Query: What problems and concerns are there in making up descriptive titles? What difficulties are involved in automatically retrieving articles from approximate titles? What is the usual relevance of the content of articles to their titles?


[ 721  428  588  602 1280  812 1298  710  835   37]
Cosine similarity  id   Document
0.3666647354      721   Information Transfer Limitations of Titles of Chemical Documents Bottle, R.R. Seeley, C.R. Some meth
0.3389680289      428   The Information Content of Titles in Engineering Literature Bottle, Robert T. Since many alerting an
0.3029264456      588   Are Titles of Chemical Papers Becoming More Informative? Tocatlian, Jacques J. The efficiency of key
0.264033071      602   The Efficiency of MEDLARS Titles for Retrieval Miller, William L. Previous research has indicated th
0.2525271318      1280   Relative Effectiveness of Document Titles and Abstracts for Determining Relevance of Documents Resni
0.2125931396      812   Com